In [3]:
import pandas as pd
import os
# from src.data_matching.main import get_edgelist

In [4]:
df_source=pd.read_csv("./tests/datasets/Wikidata/Musicians/Musicians_joinable/musicians_joinable_source.csv")
df_target=pd.read_csv("./tests/datasets/Wikidata/Musicians/Musicians_joinable/musicians_joinable_source.csv")
df_source.columns,df_target.columns

(Index(['musician', 'birthDate', 'familyNameLabel', 'givenNameLabel',
        'numberOfChildren', 'websiteLabel', 'residenceLabel', 'ethnicityLabel',
        'religionLabel', 'activityStart', 'twitterNameLabel', 'geniusNameLabel',
        'recordLabelLabel'],
       dtype='object'),
 Index(['musician', 'birthDate', 'familyNameLabel', 'givenNameLabel',
        'numberOfChildren', 'websiteLabel', 'residenceLabel', 'ethnicityLabel',
        'religionLabel', 'activityStart', 'twitterNameLabel', 'geniusNameLabel',
        'recordLabelLabel'],
       dtype='object'))

In [2]:
path="/mnt/c/Users/nkired/Documents/data-matching/tests/input_data"
# for file in os.listdir(path):
#     base_name=os.path.basename(file).split(".")[0]
#     path_file=os.path.join(path,file)
#     df=pd.read_csv(path_file)
#     reduced_df=df.sample(int(0.25*len(df)))
#     base_new_name=base_name+"_small.csv"
#     reduced_df.to_csv(os.path.join(path,base_new_name),index=False)


In [3]:
from src.data_matching.EmbDI.edgelist import EdgeList

def get_edgelist(input_file, out_dir, export: bool =True, overwrite: bool = True, dry_run: bool = False):
    """Translate an input CSV file into an edgelist."""
    dfpath = input_file
    base_name = os.path.basename(input_file).replace(".csv", ".txt")
    edgefile = os.path.join(out_dir, base_name)
    info_file = None
    df = pd.read_csv(dfpath, low_memory=False)
    pref = ["3#__tn", "3$__tt", "5$__idx", "1$__cid"]
    el = EdgeList(df, edgefile, pref, info_file, flatten=True)
    if dry_run:
        if export:
            el.convert_to_dict()
            gdict = el.convert_to_dict()
            print("el", el.convert_to_dict())
            print("gdict", gdict)
            g_nx = nx.from_dict_of_lists(gdict)
            n, _ = osp.splitext(edgefile)
            nx_fname = n + ".nx"
            pkl_fname = n + ".pkl"
            if overwrite:
                with open(nx_fname, "wb") as nx_file:
                    pickle.dump(g_nx, nx_file)
                with open(pkl_fname, "wb") as pkl_file:
                    pickle.dump(gdict, pkl_file)


In [4]:
input_path="/mnt/c/Users/nkired/Documents/data-matching/tests/input_data"
output_path="/mnt/c/Users/nkired/Documents/data-matching/tests/output_data"
for file in os.listdir(input_path):
    path_file=os.path.join(input_path,file)
    if "small" in file:
        get_edgelist(path_file, output_path)
        

340it [00:00, 2333.50it/s]


806it [00:00, 2619.71it/s]


1086it [00:00, 2388.34it/s]


750it [00:00, 1710.18it/s]


654it [00:00, 1765.01it/s]


...

13980it [00:04, 3064.97it/s]


1250it [00:00, 1528.78it/s]


10it [00:00, 948.14it/s]


5813it [00:04, 1354.35it/s]


2906it [00:02, 1390.19it/s]

In [68]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
# model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

In [69]:
from transformers import BartConfig, BartModel

# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig(d_model=16)

# Initializing a model (with random weights) from the facebook/bart-large style configuration
model = BartModel(configuration)

# Accessing the model configuration
configuration = model.config

In [72]:
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
text="football"  
text2="breoherjnmvjenje"  
tokens2 = tokenizer.encode(text2, add_special_tokens=True)
tokens = tokenizer.encode(text, add_special_tokens=True)
input_ids = torch.tensor(tokens).unsqueeze(0) 
input_ids2 = torch.tensor(tokens2).unsqueeze(0) 
embeddings = model(input_ids)[0]
embeddings2 = model(input_ids2)[0]
s=np.array(list(embeddings.detach().numpy())[0][1]).reshape(1,-1)
s2=np.array(list(embeddings2.detach().numpy())[0][1]).reshape(1,-1)
cosine_similarity(s, s2)

array([[0.9521549]], dtype=float32)

In [73]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method
embeddings = model.encode(['How is the weather today?', 'What is the current weather like today?'])
print(cos_sim(embeddings[0], embeddings[1]))


(…)ings-v2-base-en/resolve/main/config.json: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.17MB/s]
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Could not locate the jinaai/jina-bert-implementation--configuration_bert.py inside jinaai/jina-embeddings-v2-base-en.


OSError: jinaai/jina-embeddings-v2-base-en does not appear to have a file named jinaai/jina-bert-implementation--configuration_bert.py. Checkout 'https://huggingface.co/jinaai/jina-embeddings-v2-base-en/main' for available files.

In [74]:
from FlagEmbedding import FlagModel
sentences_1 = ["heey my name is Nour"]
sentences_2 = ["name"]
model = FlagModel('BAAI/bge-large-zh-v1.5', 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

# for s2p(short query to long passage) retrieval task, suggest to use encode_queries() which will automatically add the instruction to each query
# corpus in retrieval task can still use encode() or encode_corpus(), since they don't need instruction
queries = ['query_1', 'query_2']
passages = ["样例文档-1", "样例文档-2"]
q_embeddings = model.encode_queries(queries)
p_embeddings = model.encode(passages)
scores = q_embeddings @ p_embeddings.T


ModuleNotFoundError: No module named 'FlagEmbedding'

In [75]:
pip install FlagEmbedding

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/7c/55/b3432f43d6d7fee999bb23a547820d74c48ec540f5f7842e41aa5d8d5f3a/datasets-2.14.6-py3-none-any.whl.metadata
  Obtaining dependency information for accelerate>=0.20.1 from https://files.pythonhosted.org/packages/13/9e/ee987874058f2d93006961f6ff49e0bcb60ab9c26709ebe06bfa8707a4d8/accelerate-0.24.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages